**CLASIFICACIÓN PARTIENDO DE DATOS NUMÉRICOS Y CATEGÓRICOS**

En este notebook vamos a tratar de clasificar los apartamentos según su precio en baratos, medios o caros. Para ello estableceremos los límites en 50€ para los baratos y 150€ para los caros.
Dicha predicción se va a hacer a partir de los datos numéricos y categóricos que tenemos en el dataset de airbnb que venimos usando en las prácticas de este Bootcamp.

En primer lugar nos descargamos el fichero de internet y lo copiamos en un directorio local de My Drive donde tenemos recogido todo el entorno de esta práctica.
También montamos el google collab con My Drive para tenerlo vinculado. 

Solo será necesario la primera vez, luego podemos bajar unas celdas más abajo para seguir con el ejercicio.

In [ ]:
# nos descargamos el dataset de OpenDataSoft
!wget -O "airbnb-listings.csv" "https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B"

!ls -lah

--2020-06-25 06:01:03--  https://public.opendatasoft.com/explore/dataset/airbnb-listings/download/?format=csv&disjunctive.host_verifications=true&disjunctive.amenities=true&disjunctive.features=true&refine.country=Spain&q=Madrid&timezone=Europe/London&use_labels_for_header=true&csv_separator=%3B
Resolving public.opendatasoft.com (public.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to public.opendatasoft.com (public.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘airbnb-listings.csv’

airbnb-listings.csv     [  <=>               ]  54.19M  2.77MB/s    in 50s     

2020-06-25 06:02:01 (1.09 MB/s) - ‘airbnb-listings.csv’ saved [56826824]

total 55M
drwxr-xr-x 1 root root 4.0K Jun 25 06:01 .
drwxr-xr-x 1 root root 4.0K Jun 25 05:59 ..
-rw-r--r-- 1 root root  55M Jun 25 06:02 airbnb-listings.csv
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwx------ 4 root root 4.0K Jun 

In [ ]:
!ls -lah

total 109M
drwxr-xr-x 1 root root 4.0K Jun 25 06:05 .
drwxr-xr-x 1 root root 4.0K Jun 25 05:59 ..
-rw-r--r-- 1 root root  55M Jun 25 06:02 airbnb-listings.csv
drwxr-xr-x 1 root root 4.0K Jun 19 16:15 .config
drwx------ 4 root root 4.0K Jun 25 06:00 drive
drwxr-xr-x 1 root root 4.0K Jun 17 16:18 sample_data
-rw-r--r-- 1 root root  11M Jun 25 06:05 test.csv
-rw-r--r-- 1 root root  35M Jun 25 06:05 train.csv
-rw-r--r-- 1 root root 8.9M Jun 25 06:05 val.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!cp airbnb-listings.csv "drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning"

Esta parte de descarga, montado y copiado solo hace falta ejecutarla la primera vez. Una vez que lo tenemos almacenado en My Drive solo necesitamos cargarlo directamente.

A partir de aquí empieza nuestro ejercicio de clasificación.

Como hábito de buena costumbre, para no incurrir en errores involuntarios, en primer lugar se va a dividir el dataset original en train, validación y test.

Se trabaja únicamente con el de train con el objetivo de elegir un modelo. Eso se verifica con el conjunto de validation y finalmente se aplica ese "entrenamiento" al bloque de test.

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



full_df = pd.read_csv('drive/My Drive/BootcampBD&ML/práctica/prácticaDeepLearning/airbnb-listings.csv', sep=';', decimal='.')
full_train, test = train_test_split(full_df, test_size=0.2, shuffle=True, random_state=0)
train, val = train_test_split(full_train, test_size=0.2, shuffle=True, random_state=0)

print(f'Dimensiones del dataset de training: {train.shape}')
print(f'Dimensiones del dataset de validación: {val.shape}')
print(f'Dimensiones del dataset de test: {test.shape}')

# Guardamos
train.to_csv('./train.csv', sep=';', decimal='.', index=True)
val.to_csv('./val.csv', sep=';', decimal='.', index=True)
test.to_csv('./test.csv', sep=';', decimal='.', index=True)


Dimensiones del dataset de training: (8960, 89)
Dimensiones del dataset de validación: (2240, 89)
Dimensiones del dataset de test: (2801, 89)


En primer lugar vamos a procesar nuestros datos de entrada con la intención de darles el formato adecuado y que nuestro modelo pueda trabajar con ellos.

Vemos que hay 89 columnas y necesitamos saber qué tipo de información disponemos. En este punto nos valemos del trabajo ya realizado en el módulo de ML. Queda explicado a continuación:

1.- Filtramos por la ciudad de Madrid. Quiero asegurarme que solo trabajamos con apartamentos de dicha ciudad.

2.- Analizamos nuestras features y decidimos si los vamos a usar en nuestro entrenamiento o si podemos prescindir de ellas. Para ello usamos los siguientes comandos: - df['xxxxx'].describe() - df['xxxxx'].value_counts()

Podemos destacar las siguientes agrupaciones de columnas:

- Columnas relacionadas con ID: considero que es información que identifica cada propiedad de forma singular y unívoca y no guarda ninguna relación con otras propiedades ni sus precios. Podemos eliminarlas de nuestro dataset. ['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped']
- Columnas relacionas con URLs: son URLs que nos direccionan a otro sitio web e incluyen fotos. En este caso no vamos a hacer tratamiento de imagen, así que podemos eliminarlas. ['Listing Url','Thumbnail Url','Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',]
- Columnas descriptivas: Son columnas de tipo objeto. Contienen mucha literatura y la información es redundante con otras más específicas. Son resúmenes detallando el tipo de propiedad y sus características (número de habitaciones, calle, barrio, etc). Se da el caso también que el número de entradas únicas para cada una de estas columnas es muy parecido al número total, por lo que además es muy complicado hacer agrupaciones o clústers. Por tanto podemos eliminarlas. ['Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access','Interaction','House Rules','Host Name','Host About','Street']
- Columnas que hacen referencia a Madrid, España: Ya hemos filtrado por la ciudad de Madrid, así que la información relacionada con País, Estado o similar no nos aporta nada. Añadimos también la columna Geolocation que está duplicada con Latitude y Longitude. Podemos eliminarlas del dataset. ['Host Location','State','Market','Smart Location','Country Code','Country','Geolocation']
- Columnas relacionadas con el precio: son columnas que nos hacen trampa a la hora de predecir el precio. Las eliminamos del dataset. ['Weekly Price','Monthly Price']
- Otras columnas que no aportan valor: aquí agrupamos columnas donde todos los valores son none, están vacías o solo tienen valores un número de entradas menor que el 5% del total del dataset. ['Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet']

3.- Analizamos la variable objetivo: Price. Vamos a ver su histograma y sus datos más relevantes:
<img src='https://docs.google.com/drawings/d/1JZwa8QDoZVXf7xI9etKMyambCuNas55_K8R7IwCvbnk/edit?usp=sharing' alt="data_blending" border="0" width="500">
![](https://drive.google.com/uc?export=view&id=1JZwa8QDoZVXf7xI9etKMyambCuNas55_K8R7IwCvbnk)
![](img/Hist Price)
<figure>
<center>
<img src='https://docs.google.com/drawings/d/1JZwa8QDoZVXf7xI9etKMyambCuNas55_K8R7IwCvbnk/edit?usp=sharing' />
<figcaption>Image Caption</figcaption></center>
</figure>

A la vista de los resultados consideramos que más de 400€ es outlier y hay 6 entradas en las que hay que imputar valores. Lo haremos usando la media. Además se aprecia que tiene una distribución logarítmica, por lo que vamos a aplicarle también dicha transformación

4.- Trabajamos con las columnas que sabemos son de tipo fecha. En primer lugar le damos formato de año-mes-día y luego lo restamos a 2017, que lo tomamos como referencia de cuando se creó el dataset. De esta forma obtenemos columnas de tipo float cuyo valor mínimo será 0 (indica que es muy reciente) y el máximo será el número de años de diferencia entre 2017 y el evento concreto (Host since o First/Last Review).

5.- Imputamos valores para completar nuestro dataset. Usamos la siguiente línea para ver donde hay campos vacíos: filtered_df.isnull().any(), y esta otra para ver el detalle de los valores: filtered_df['xxxx'].describe()

6.- Analizamos variables descriptivas y contamos palabras. Este método es algo muy básico y no nos aporta una información precisa. Queda pendiente de mejora con técnicas NLP.

7.- Analizamos las columnas que todavía no son numéricas. La idea es transformarlas o codificarlas con un MeanEncoder. Utilizamos el comando filtered_df.dtypes para comprobarlo.

8.- Eliminimos la variable Price ya que está categorizada y si lo usamos como entrada nos falsearía el trabajo de predicción que pretendemos hacer.

9.- Damos formato a nuestro dataset para devolver por un lado las variables de entrada X y la variable objetivo Y.

10.- Escalamos los valores para usar valores entre 0 y 1 y así obtener mejor resultados con nuestros modelos.


Es importante destacar que todas esas transformaciones las hacemos en nuestros tres conjuntos de trabajo para ya tenerlos preparados. Tomamos siempre como referencia los valores obtenidos en nuestro conjuntos de train y los aplicamos a los valores de validation y test.

In [ ]:
# A partir de este momento cargamos el dataset de train y trabajamos ÚNICAMENTE con él. 
df_train = pd.read_csv('./train.csv', sep=';', decimal='.')
df_val = pd.read_csv('./val.csv', sep=';', decimal='.')
df_test = pd.read_csv('./test.csv', sep=';', decimal='.')

def preprocesado(train, val, test):
  #Nos quedamos solo con las filas que pertenecen a la ciudad de Madrid
  indexNames = train[ train['City'] != 'Madrid' ].index
  train.drop(indexNames , inplace=True)
  train.drop(['City'], axis=1, inplace=True)

  indexNames = val[ val['City'] != 'Madrid' ].index
  val.drop(indexNames , inplace=True)
  val.drop(['City'], axis=1, inplace=True)

  indexNames = test[ test['City'] != 'Madrid' ].index
  test.drop(indexNames , inplace=True)
  test.drop(['City'], axis=1, inplace=True)

  #eliminamos las columnas que no aportan
  train.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet'], 
        axis=1, inplace=True)
  val.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet'], 
        axis=1, inplace=True)
  test.drop(['ID','Scrape ID','Last Scraped','Host ID','Calendar last Scraped','Listing Url','Thumbnail Url',
         'Medium Url','Picture Url','XL Picture Url','Host URL','Host Thumbnail Url','Host Picture Url',
        'Name','Summary','Space','Description','Neighborhood Overview','Notes','Transit','Access',
         'Interaction','House Rules','Host Name','Host About','Street','Host Location','State','Market',
         'Smart Location','Country Code','Country','Geolocation','Weekly Price','Monthly Price',
         'Host Acceptance Rate','Experiences Offered','Has Availability','License','Jurisdiction Names','Square Feet'], 
        axis=1, inplace=True)
  
  #nueva variable --> en DL no lo uso (esta variable viene de ML) porque hemos visto que nos baja el accuracy
  #train['Bed_Bath_Rooms'] = train['Bedrooms']*train['Bathrooms']
  #val['Bed_Bath_Rooms'] = val['Bedrooms']*val['Bathrooms']
  #test['Bed_Bath_Rooms'] = test['Bedrooms']*test['Bathrooms']
  
  #PRICE
  #imputamos valores vacíos con la media de train
  MeanPriceTrain = train['Price'].mean()
  train['Price'].fillna(MeanPriceTrain, inplace=True)
  val['Price'].fillna(MeanPriceTrain, inplace=True)
  test['Price'].fillna(MeanPriceTrain, inplace=True)
  #definimos outlier >400€
  Price_filter = train['Price'] <= 400
  filtered_train = train[Price_filter]
  Price_filter = val['Price'] <= 400
  filtered_val = val[Price_filter]
  Price_filter = test['Price'] <= 400
  filtered_test = test[Price_filter]
  #transformamos variable Price a gausiana
  filtered_train['Price'] = filtered_train['Price'].apply(lambda x: np.log10(x))
  filtered_val['Price'] = filtered_val['Price'].apply(lambda x: np.log10(x))
  filtered_test['Price'] = filtered_test['Price'].apply(lambda x: np.log10(x))
  #categorizamos la variable precio en 3 tipos: barato (0), medio (1) y caro (2).
  filtered_train['Cat_Price'] = filtered_train['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  filtered_val['Cat_Price'] = filtered_val['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  filtered_test['Cat_Price'] = filtered_test['Price'].apply(lambda x: 0 if x < np.log10(50) else (1 if x < np.log10(150) else 2))
  
  
  #FECHAS
  filtered_train['Host Since'] = pd.to_datetime(filtered_train['Host Since'], format="%Y-%m-%d")
  filtered_train['First Review'] = pd.to_datetime(filtered_train['First Review'], format="%Y-%m-%d")
  filtered_train['Last Review'] = pd.to_datetime(filtered_train['Last Review'], format="%Y-%m-%d")
  filtered_train['Host Since'] = filtered_train['Host Since'].apply(lambda x: 2017 - x.year)
  filtered_train['First Review'] = filtered_train['First Review'].apply(lambda x: 2017 - x.year)
  filtered_train['Last Review'] = filtered_train['Last Review'].apply(lambda x: 2017 - x.year)

  filtered_val['Host Since'] = pd.to_datetime(filtered_val['Host Since'], format="%Y-%m-%d")
  filtered_val['First Review'] = pd.to_datetime(filtered_val['First Review'], format="%Y-%m-%d")
  filtered_val['Last Review'] = pd.to_datetime(filtered_val['Last Review'], format="%Y-%m-%d")
  filtered_val['Host Since'] = filtered_val['Host Since'].apply(lambda x: 2017 - x.year)
  filtered_val['First Review'] = filtered_val['First Review'].apply(lambda x: 2017 - x.year)
  filtered_val['Last Review'] = filtered_val['Last Review'].apply(lambda x: 2017 - x.year)

  filtered_test['Host Since'] = pd.to_datetime(filtered_test['Host Since'], format="%Y-%m-%d")
  filtered_test['First Review'] = pd.to_datetime(filtered_test['First Review'], format="%Y-%m-%d")
  filtered_test['Last Review'] = pd.to_datetime(filtered_test['Last Review'], format="%Y-%m-%d")
  filtered_test['Host Since'] = filtered_test['Host Since'].apply(lambda x: 2017 - x.year)
  filtered_test['First Review'] = filtered_test['First Review'].apply(lambda x: 2017 - x.year)
  filtered_test['Last Review'] = filtered_test['Last Review'].apply(lambda x: 2017 - x.year)

  #Imputamos valores en variables categóricas donde tomamos la moda para los valores que faltan.
  #Lo extraemos en una variable disinta para cada columna con la intención de aplicar el mismo valor en val y test
  ModeHSTrain = filtered_train['Host Since'].mode()[0]
  ModeHLCTrain = filtered_train['Host Listings Count'].mode()[0]
  ModeHTLCTrain = filtered_train['Host Total Listings Count'].mode()[0]
  ModeBathroomsTrain = filtered_train['Bathrooms'].mode()[0]
  ModeBedroomsTrain = filtered_train['Bedrooms'].mode()[0]
  ModeBedsTrain = filtered_train['Beds'].mode()[0]

  filtered_train['Host Since'].fillna(ModeHSTrain, inplace=True)
  filtered_train['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  filtered_train['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  filtered_train['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  filtered_train['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  filtered_train['Beds'].fillna(ModeBedsTrain, inplace=True)

  filtered_val['Host Since'].fillna(ModeHSTrain, inplace=True)
  filtered_val['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  filtered_val['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  filtered_val['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  filtered_val['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  filtered_val['Beds'].fillna(ModeBedsTrain, inplace=True)
  filtered_test['Host Since'].fillna(ModeHSTrain, inplace=True)
  filtered_test['Host Listings Count'].fillna(ModeHLCTrain, inplace=True)
  filtered_test['Host Total Listings Count'].fillna(ModeHTLCTrain, inplace=True)
  filtered_test['Bathrooms'].fillna(ModeBathroomsTrain, inplace=True)
  filtered_test['Bedrooms'].fillna(ModeBedroomsTrain, inplace=True)
  filtered_test['Beds'].fillna(ModeBedsTrain, inplace=True)

  #Imputamos valores en variables lineales donde tomamos la media para los valores que faltan
  #Lo extraemos en una variable disinta para cada columna con la intención de aplicar el mismo valor en val y test
  MeanRSRatingTrain = filtered_train['Review Scores Rating'].mean()
  MeanRSAccuracyTrain = filtered_train['Review Scores Accuracy'].mean()
  MeanRSCleanlinessTrain = filtered_train['Review Scores Cleanliness'].mean()
  MeanRSCheckinTrain = filtered_train['Review Scores Checkin'].mean()
  MeanRSCommunicationTrain = filtered_train['Review Scores Communication'].mean()
  MeanRSLocationTrain = filtered_train['Review Scores Location'].mean()
  MeanRSValueTrain = filtered_train['Review Scores Value'].mean()

  filtered_train['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  filtered_train['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  filtered_train['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  filtered_train['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  filtered_train['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  filtered_train['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  filtered_train['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)
  filtered_val['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  filtered_val['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  filtered_val['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  filtered_val['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  filtered_val['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  filtered_val['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  filtered_val['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)
  filtered_test['Review Scores Rating'].fillna(MeanRSRatingTrain, inplace=True)
  filtered_test['Review Scores Accuracy'].fillna(MeanRSAccuracyTrain, inplace=True)
  filtered_test['Review Scores Cleanliness'].fillna(MeanRSCleanlinessTrain, inplace=True)
  filtered_test['Review Scores Checkin'].fillna(MeanRSCheckinTrain, inplace=True)
  filtered_test['Review Scores Communication'].fillna(MeanRSCommunicationTrain, inplace=True)
  filtered_test['Review Scores Location'].fillna(MeanRSLocationTrain, inplace=True)
  filtered_test['Review Scores Value'].fillna(MeanRSValueTrain, inplace=True)

  #los vacíos los consideramos como desconocidos
  filtered_train['Host Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_train['Host Verifications'].fillna('Unknown', inplace=True)
  filtered_train['Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_train['Zipcode'].fillna('Unknown', inplace=True)
  filtered_train['Amenities'].fillna('Unknown', inplace=True)
  filtered_train['First Review'].fillna('Unknown', inplace=True)
  filtered_train['Last Review'].fillna('Unknown', inplace=True)
  filtered_val['Host Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_val['Host Verifications'].fillna('Unknown', inplace=True)
  filtered_val['Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_val['Zipcode'].fillna('Unknown', inplace=True)
  filtered_val['Amenities'].fillna('Unknown', inplace=True)
  filtered_val['First Review'].fillna('Unknown', inplace=True)
  filtered_val['Last Review'].fillna('Unknown', inplace=True)
  filtered_test['Host Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_test['Host Verifications'].fillna('Unknown', inplace=True)
  filtered_test['Neighbourhood'].fillna('Unknown', inplace=True)
  filtered_test['Zipcode'].fillna('Unknown', inplace=True)
  filtered_test['Amenities'].fillna('Unknown', inplace=True)
  filtered_test['First Review'].fillna('Unknown', inplace=True)
  filtered_test['Last Review'].fillna('Unknown', inplace=True)

  #consideramos que donde falta un valor es porque no existe, es decir, no hay respuesta o la tasa es 0€
  filtered_train['Host Response Time'].fillna('No response', inplace=True)
  filtered_train['Host Response Rate'].fillna(0, inplace=True)
  filtered_train['Security Deposit'].fillna(0, inplace=True)
  filtered_train['Cleaning Fee'].fillna(0, inplace=True)
  filtered_train['Reviews per Month'].fillna(0, inplace=True)
  filtered_val['Host Response Time'].fillna('No response', inplace=True)
  filtered_val['Host Response Rate'].fillna(0, inplace=True)
  filtered_val['Security Deposit'].fillna(0, inplace=True)
  filtered_val['Cleaning Fee'].fillna(0, inplace=True)
  filtered_val['Reviews per Month'].fillna(0, inplace=True)
  filtered_test['Host Response Time'].fillna('No response', inplace=True)
  filtered_test['Host Response Rate'].fillna(0, inplace=True)
  filtered_test['Security Deposit'].fillna(0, inplace=True)
  filtered_test['Cleaning Fee'].fillna(0, inplace=True)
  filtered_test['Reviews per Month'].fillna(0, inplace=True)

  #transformaciones contando palabras. es algo muy sencillo, queda pendiente mejorarlo con técnicas NLP en el futuro
  filtered_train['Amenities'] = filtered_train['Amenities'].apply(lambda x: len(str(x).split(',')))
  filtered_train['Host Verifications'] = filtered_train['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  filtered_train['Features'] = filtered_train['Features'].apply(lambda x: len(str(x).split(',')))
  filtered_val['Amenities'] = filtered_val['Amenities'].apply(lambda x: len(str(x).split(',')))
  filtered_val['Host Verifications'] = filtered_val['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  filtered_val['Features'] = filtered_val['Features'].apply(lambda x: len(str(x).split(',')))
  filtered_test['Amenities'] = filtered_test['Amenities'].apply(lambda x: len(str(x).split(',')))
  filtered_test['Host Verifications'] = filtered_test['Host Verifications'].apply(lambda x: len(str(x).split(',')))
  filtered_test['Features'] = filtered_test['Features'].apply(lambda x: len(str(x).split(',')))

  #MeanEncoder
  categorical = ['Host Response Time', 'Host Neighbourhood', 'Neighbourhood','Neighbourhood Cleansed',
               'Neighbourhood Group Cleansed','Zipcode','Property Type','Room Type','Bed Type',
               'Calendar Updated','First Review','Last Review','Cancellation Policy']
  # En train creamos un dict para usarlo después en val y test
  mean_map = {}
  for c in categorical:
      mean = filtered_train.groupby(c)['Price'].mean()
      filtered_train[c] = filtered_train[c].map(mean)    
      mean_map[c] = mean
  for c in categorical:
    filtered_val[c] = filtered_val[c].map(mean_map[c])
  for c in categorical:
    filtered_test[c] = filtered_test[c].map(mean_map[c])
 #los valores vacíos de val y test los completo con la moda de train
  for c in categorical:
    filtered_val[c].fillna(filtered_train[c].mode()[0], inplace=True)
  for c in categorical:
    filtered_test[c].fillna(filtered_train[c].mode()[0], inplace=True)

  #eliminamos la variable Price (ya está categorizada y ya se ha usado para el MeanEncoder)
  filtered_train.drop(['Price'],axis=1, inplace=True)
  filtered_val.drop(['Price'],axis=1, inplace=True)
  filtered_test.drop(['Price'],axis=1, inplace=True)
  
  #separamos las variables de entrada de la variable objetivo
  cols = filtered_train.columns.tolist()
  Xtrain = filtered_train[cols[0:-1]]
  Xval = filtered_val[cols[0:-1]]
  Xtest = filtered_test[cols[0:-1]]

  #escalamos los valores de entrada
  cs = MinMaxScaler()
  Xtrain_Scaled = cs.fit_transform(Xtrain)
  Xval_Scaled = cs.transform(Xval)
  Xtest_Scaled = cs. transform(Xtest)

  #extraemos la variable objetivo
  Ytrain = filtered_train[cols[-1]]
  Yval = filtered_val[cols[-1]]
  Ytest = filtered_test[cols[-1]]

  return (Xtrain_Scaled, Xval_Scaled, Xtest_Scaled, Ytrain, Yval, Ytest)
      
  





Usamos la función definida previamente para obtener nuestros conjuntos de datos y la variable objetivo.

In [ ]:
(Xtrain, Xval, Xtest, ytrain, yval, ytest) = preprocesado(df_train, df_val, df_test)

Ahora vamos a definir los modelos con los que vamos a trabajar y que iremos comparando.

In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Dense, Flatten, Input

#creamos una primera red muy sencilla con una sola capa de entrada, otra oculta de 4 neuronas y otra de clasificación
def MiRed1(dim):
  model = Sequential()
  model.add(Dense(8, input_dim=dim, activation="relu"))
  model.add(Dense(4, activation="relu"))
  model.add(Dense(3, activation="softmax"))

  return model

#creamos una red un poco más compleja con más capas ocultas
def MiRed2(dim):
  model = Sequential()
  model.add(Dense(64, input_dim=dim, activation="relu"))
  model.add(Dense(32, activation="relu"))
  model.add(Dense(16, activation="relu"))
  model.add(Dense(8, activation="relu"))
  model.add(Dense(4, activation="relu"))
  model.add(Dense(3, activation="softmax"))

  return model

Empezamos usando el modelo 1:

In [ ]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.optimizers import Adam

#categorización one-hot
num_classes = 3
Ytrain = to_categorical(ytrain, num_classes)
Yval = to_categorical(yval, num_classes)
Ytest = to_categorical(ytest, num_classes)

model = MiRed1(Xtrain.shape[1])
opt = Adam(lr=1e-3, decay=1e-3 / 200)

#compilamos el modelo
model.compile(loss="categorical_crossentropy", 
              optimizer=opt,
							metrics=['accuracy'])

# entrenamos el modelo
print("[INFO] training model...")
model.fit(x=Xtrain, y=Ytrain, 
	validation_data=(Xval, Yval),
	epochs=100, batch_size=8)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


[INFO] training model...

Train on 8412 samples, validate on 2100 samples
Epoch 1/100
8412/8412 [==============================] - 5s 575us/step - loss: 0.7049 - accuracy: 0.7184 - val_loss: 0.5904 - val_accuracy: 0.7952
Epoch 2/100
8412/8412 [==============================] - 3s 368us/step - loss: 0.5277 - accuracy: 0.8084 - val_loss: 0.5358 - val_accuracy: 0.7995
Epoch 3/100
8412/8412 [==============================] - 3s 378us/step - loss: 0.4911 - accuracy: 0.8113 - val_loss: 0.4981 - val_accuracy: 0.8029
Epoch 4/100
8412/8412 [==============================] - 3s 383us/step - loss: 0.4693 - accuracy: 0.8125 - val_loss: 0.4807 - val_accuracy: 0.8076
Epoch 5/100
8412/8412 [==============================] - 3s 381us/step - loss: 0.4534 - accuracy: 0.8181 - val_loss: 0.4780 - val_accuracy: 0.8114
Epoch 6/100
8412/8412 [==============================] - 3s 376us/step - loss: 0.4448 - accuracy: 0.8239 - val_loss: 0.4740 - val_accuracy: 0.8124
Epoch 7/100
8412/8412 [=====================

Ahora usamos el modelo 2:

In [ ]:
model2 = MiRed2(Xtrain.shape[1])
opt = Adam(lr=1e-3, decay=1e-3 / 200)

#compilamos el modelo
model2.compile(loss="categorical_crossentropy",
                optimizer=opt,
								metrics=['accuracy'])

# entrenamos el modelo
print("[INFO] training model...")
model2.fit(x=Xtrain, y=Ytrain, 
	validation_data=(Xval, Yval),
	epochs=100, batch_size=8)

[INFO] training model...
Train on 8412 samples, validate on 2100 samples
Epoch 1/100
8412/8412 [==============================] - 5s 541us/step - loss: 0.7000 - accuracy: 0.7688 - val_loss: 0.5665 - val_accuracy: 0.8067
Epoch 2/100
8412/8412 [==============================] - 4s 514us/step - loss: 0.5172 - accuracy: 0.8142 - val_loss: 0.4999 - val_accuracy: 0.8148
Epoch 3/100
8412/8412 [==============================] - 4s 524us/step - loss: 0.4702 - accuracy: 0.8201 - val_loss: 0.4788 - val_accuracy: 0.8048
Epoch 4/100
8412/8412 [==============================] - 4s 488us/step - loss: 0.4545 - accuracy: 0.8258 - val_loss: 0.4754 - val_accuracy: 0.8190
Epoch 5/100
8412/8412 [==============================] - 4s 523us/step - loss: 0.4444 - accuracy: 0.8267 - val_loss: 0.4740 - val_accuracy: 0.8186
Epoch 6/100
8412/8412 [==============================] - 4s 496us/step - loss: 0.4365 - accuracy: 0.8338 - val_loss: 0.4601 - val_accuracy: 0.8195
Epoch 7/100
8412/8412 [======================

Como podemos ver en los resultados de las ejecuciones anteriores tenemos dos modelos:

- Modelo 1: Se trata de una red sencilla con una sola capa de entrada de 8 neuronas, otra oculta de 4 y una de clasificación de salida con 3 neuronas (nuestras 3 posibles salidas).
- Modelo 2: Es una red algo más compleja con más capas ocultas. Empezamos con una de 64 neuronas y vamos bajando en potencias de 2 hasta llegar a 4. La capa de clasificación de salida es igual que el caso anterior.

Ambos modelos los comparamos en igualdad de condiciones, es decir, con el mismo learning rate, batch_size, igual número de épocas... y vemos que el modelo1 tiene mejores prestaciones:
- Accuracy: El modelo 1 tiene un 84% en train y un 83% en val. Mientras que el modelo 2 tiene un 90% en train y un 83% en val. Eso nos indica overfitting en el modelo 2 (se aprende muy bien el conjunto de train y luego no es capaz de generalizarlo tan bien en el modelo de validación) y puede venir dado por la complejidad de esta segunda red.
- Tiempo de ejecución: En el modelo 1 cada época tarda 3 segundos, mientras que en el segundo modelo tarda 4 ó 5. Es decir, es más rápido el primero.

Por tanto, nos quedamos con el modelo 1.

In [ ]:
Xtrain.shape

(8412, 47)

Quiero comentar un caso curioso que me ha pasado entrenando estos modelos. En una primera instancia el conjunto de datos de entrada tenía 48 columnas (finalmente se ha quedado en 47). Los resultados obtenidos eran mucho peores y un tanto desconcertantes. 

El modelo parecía que a partir de la segunda época ya había convergido y se quedaba con unos valores de accuracy de 50% en train y 47% en val. 
Realmente era como si el modelo no aprendiera, así que después de verificar que las etiquetas estaban bien comencé a entrenar el modelo solo con unas cuantas variables de entrada y los resultados fueron mejorando. El accuracy empezó a subir según iba incrementando el número de variables de entrada. Hay que tener en cuenta que para este proceso no se tenía en cuenta qué varibales de entrada se usaban, pero sí se observaba que el comportamiento de los modelos ya era más coherente. 

Llegué a la conclusión que la variable que estropeaba el funcionamiento era la última ya que con 47 variables de entrada obtenemos los resultados que hemos comentado antes superiores al 80% y con 48 bajábamos al 50%.
Esa variable en cuestión es una que yo creaba manualmente (viene heredado del módulo de ML) donde multiplicaba el número de baños por el número de habitaciones.

Para terminar evaluamos el modelo elegido con el conjunto de test:

In [ ]:
# Evaluamos el modelo
scores = model.evaluate(Xtest, Ytest)

print('Test Loss: %.3f' % scores[0])
print('Test Accuracy: %.3f' % scores[1])

2642/2642 [==============================] - 0s 45us/step
Test Loss: 0.393
Test Accuracy: 0.846
